In [16]:
import pandas as pd
import json
import numpy as np
from collections import defaultdict
exact_result = {}
exact_runtime = defaultdict(list)
multi_table_runtime = defaultdict(list)
single_table_runtime = defaultdict(list)
multi_table_results = defaultdict(list)
single_table_results = defaultdict(list)
with open("all_results.jsonl") as f:
    for line in f:
        result = json.loads(line)
        query_id = result["query_id"]
        if "multi_table_sample_time" in result and result["multi_table_sample_time"] != -1:
            multi_table_runtime[query_id].append(result["multi_table_sample_time"] + result["pilot_time"])
            multi_table_results[query_id].append(result["multi_table_sample_result"])
        elif "single_table_sample_time" in result and result["single_table_sample_time"] != -1:
            single_table_runtime[query_id].append(result["single_table_sample_time"] + result["pilot_time"])
            single_table_results[query_id].append(result["single_table_sample_result"])
        elif "exact_time" in result:
            exact_runtime[query_id].append(result["exact_time"])
            exact_result[query_id] = result["exact_result"]
print("multi_time", multi_table_runtime)
print("single_time", single_table_runtime)
        # # find the column is not of string type
        # print(query_id)
        # if multi_result.empty:
        #     print("multi_result is empty")
        # else:
        #     non_string_columns = []
        #     for column in multi_result.columns:
        #         if single_result[column].dtype != "object":
        #             non_string_columns.append(column)
        #     error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
        #     max_error = error.max().max()
        #     print("multi table sample error: ", max_error*100)
        # if single_result.empty:
        #     print("single_result is empty")
        # else:
        #     non_string_columns = []
        #     for column in single_result.columns:
        #         if single_result[column].dtype != "object":
        #             non_string_columns.append(column)
        #     error = np.abs(single_result[non_string_columns] - exact_result[non_string_columns]) / exact_result[non_string_columns]
        #     max_error = error.max().max()
        #     print("single table sample error: ",max_error*100)


multi_time defaultdict(<class 'list'>, {5: [18.067424058914185, 19.811666011810303, 19.581096649169922], 7: [106.8329381942749, 111.94026708602905], 8: [42.890650033950806, 21.094711542129517, 20.837751626968384], 9: [97.80574440956116, 94.48318648338318, 97.23134112358093], 12: [9.55010986328125, 7.313006401062012, 7.365624666213989], 14: [12.672680377960205, 12.806903600692749, 12.815322399139404], 19: [53.86165523529053, 50.928062438964844, 50.42456531524658]})
single_time defaultdict(<class 'list'>, {5: [15.56600832939148, 14.237381219863892, 13.887486934661865], 7: [119.80557322502136, 120.21138453483582, 116.67276930809021], 9: [90.23223447799683, 84.33879399299622, 83.01623821258545], 12: [4.2511842250823975, 4.176263809204102, 4.213051080703735], 14: [13.460443496704102, 13.4953031539917, 13.013299703598022], 19: [50.03378987312317, 52.65067005157471, 51.67769265174866]})


In [17]:
for query_id in exact_result:
    print("query id", query_id)
    exact_runtime_avg = np.mean(exact_runtime[query_id])
    exact_result_query = pd.DataFrame(exact_result[query_id])
    if query_id in multi_table_runtime:
        multi_runtime_avg = np.mean(multi_table_runtime[query_id])
        speedup = exact_runtime_avg / multi_runtime_avg
        print("multi table sample speedup: ", speedup)
        for result in multi_table_results[query_id]:
            result = pd.DataFrame(result)
            non_string_columns = []
            for column in exact_result_query:
                if result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(result[non_string_columns] - exact_result_query[non_string_columns]) / exact_result_query[non_string_columns]
            max_error = error.max().max()
            print("multi table sample error: ", max_error*100)
    if query_id in single_table_runtime:
        single_runtime_avg = np.mean(single_table_runtime[query_id])
        speedup = exact_runtime_avg / single_runtime_avg
        print("single table sample speedup: ", speedup)
        for result in single_table_results[query_id]:
            result = pd.DataFrame(result)
            non_string_columns = []
            for column in exact_result_query:
                if result[column].dtype != "object":
                    non_string_columns.append(column)
            error = np.abs(result[non_string_columns] - exact_result_query[non_string_columns]) / exact_result_query[non_string_columns]
            max_error = error.max().max()
            print("single table sample error: ", max_error*100)


query id 5
multi table sample speedup:  1.0862128033423104
multi table sample error:  0.8950724511064511
multi table sample error:  1.30116362689069
multi table sample error:  2.369940067737506
single table sample speedup:  1.4285360130178189
single table sample error:  99.9899905432412
single table sample error:  99.99006943607446
single table sample error:  99.99015057340938
query id 7
multi table sample speedup:  1.092815405284268
multi table sample error:  1.911230209928761
multi table sample error:  1.0382854910724069
single table sample speedup:  1.0054062852847059
single table sample error:  2.541454210297663
single table sample error:  3.1228659173676525
single table sample error:  3.9577102197013
query id 8
multi table sample speedup:  0.8248363779039534
multi table sample error:  1.6795239241950712
multi table sample error:  0.5498610995558624
multi table sample error:  1.3421918200756568
query id 9
multi table sample speedup:  0.9119905883382602
multi table sample error:  2.